# Usage

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import scipy
import seaborn as sns
import skimage
import skimage.io
import tifffile
from pympler.asizeof import asizeof

from nima import nima, segmentation

cm = plt.cm.inferno_r

fp = "../../tests/data/1b_c16_15.tif"

channels = ["G", "R", "C"]
# dark = io.imread('/home/dati/GBM_persson/analyses/15.02.05_cal-GBM5-pBJclop/dark/dark-25_500.tif')
# flat = io.imread('/home/dati/GBM_persson/analyses/15.02.05_cal-GBM5-pBJclop/flat/flat-C-dark-37bis_500.tif')

In [ ]:
bg_params = segmentation.BgParams(kind="li_adaptive")
bg_params

In [ ]:
d_im, n_ch, times = nima.read_tiff(fp, channels)

In [ ]:
d_im_bg, bgs, ff = nima.d_bg(d_im, downscale=(2, 2), bg_params=bg_params)
bgs

In [ ]:
d_im_bg["G"].__sizeof__() / 1024**2

In [ ]:
ff["C"][1][0]

In [ ]:
nima.d_mask_label(
    d_im_bg, threshold_method="yen", min_size=2000, channels=channels, watershed=0
)
d_im_bg.keys()

In [ ]:
f = nima.d_show(d_im_bg)

In [ ]:
meas, pr = nima.d_meas_props(d_im_bg)
f = nima.d_plot_meas(bgs, meas, channels=channels)

In [ ]:
plt.subplot(1, 2, 1)
skimage.io.imshow(d_im_bg["r_cl"][2], vmin=0.0, vmax=1.1)
plt.subplot(1, 2, 2)
skimage.io.imshow(d_im_bg["r_pH"][2], vmin=7.3, vmax=10.7)

## dev

In [ ]:
c0 = d_im["C"][0]
g0 = d_im["G"][0]
r0 = d_im["R"][0]
tifffile.imshow(1 / c0 / c0)

In [ ]:
bg_params = segmentation.BgParams(kind="li_adaptive", erosion_disk=0)
bg_params

In [ ]:
im = c0
im = d_im["C"][1]
bg_params.adaptive_radius = int(im.shape[1] / 2)
if bg_params.adaptive_radius % 2 == 0:  # sk >0.12.0 check for even value
    bg_params.adaptive_radius += 1
bg_params

In [ ]:
bg_params.clip = False
m, title, lim = segmentation._bg_li_adaptive(im, bg_params=bg_params)
title, lim

In [ ]:
scipy.stats.distributions.norm.fit(im[m])

In [ ]:
sns.histplot(im[m], kde=True, stat="density", log_scale=(False, True))

In [ ]:
fig = plt.hist(im[m], histtype="step", bins=32, log=1)
fig = plt.hist(im[m], histtype="step", bins=32, log=1)

In [ ]:
segmentation.calculate_bg_iteratively(im, probplot=True)

## background, AF, target cells

In [ ]:
fp_mdtest = "/home/dati/GBM_persson/data/15.02.05_cal-GBM5-pBJclop/ph633/1_20_40.tif"
fp_mdtest = "/home/dati/dt-clop3/data/ionofori assay pH6.8 22.05.05/Clop FLATxy Gexp20ms Rexp3ms.tf8"

In [ ]:
nima.read_tiff(fp_mdtest, ["G", "C", "R"])

In [ ]:
dim, md = nima.read_tiffmd(fp_mdtest, channels)

In [ ]:
md.stage_position

In [ ]:
im = dim[:, 0].mean(axis=0).compute()
plt.imshow(im)
plt.colorbar()

In [ ]:
flat_im = scipy.ndimage.gaussian_filter(im, 150)
plt.imshow(flat_im)
plt.colorbar()

In [ ]:
plt.plot(flat_im[1060, :])

In [ ]:
fpm = "/home/dan/workspace/nima_io/tests/data/t4_1.tif"

In [ ]:
nima.read_tiffmd(fpm, ["G", "R2", "R", "C"])

In [ ]:
nima.read_tiffmd(fpm, ["G", "R2", "R", "C"])

In [ ]:
from nima import utils

In [ ]:
bgd = {}
for i, c in enumerate(channels):
    bgd[c] = [
        segmentation.calculate_bg_iteratively(dim[t, i].compute())
        for t in range(md.size_t[0])
    ]

bgd

In [ ]:
for t in range(md.size_t[0]):
    dim[t, 0] -= bgd[channels[0]][t].bg

In [ ]:
plt.imshow(dim[0, 0])
plt.colorbar()

In [ ]:
segmentation.calculate_bg_iteratively(dim[0, 0].compute(), probplot=1)

In [ ]:
bgd[channels[0]][1].bg

In [ ]:
t = 0

plt.imshow(dim[t, 0] - bgd[channels[0]][t].bg)
plt.colorbar()

In [ ]:
m = segmentation.prob(dim[3, 0].compute(), 0, bgd[channels[0]][3].sd * 13) > 0.001
plt.imshow(~m)

In [ ]:
asizeof(dim) / 1024**2

In [ ]:
fpm = "/home/dan/workspace/nima_io/tests/data/t4_1.tif"
tifffile.TiffFile(fpm).series

In [ ]:
md = nima.Metadata(tifffile.TiffReader(fpm))
md

In [ ]:
nima.read_tiffmd(fpm, ["G", "R2", "C", "R"])

In [ ]:
asizeof(dim) / 1024**2

In [ ]:
bgr = segmentation.calculate_bg_iteratively(dim[1, 0].compute())
bgr.bg

In [ ]:
dim, md = nima.read_tiffmd(fp_mdtest, channels=["G", "R2", "C", "R"])

md

In [ ]:
channels3 = ["G", "R", "C", "G2"]
dim3, _, _ = nima.read_tiff(fp_mdtest, channels3)

In [ ]:
dim3, bgv3, bgf3 = nima.d_bg(
    dim3, segmentation.BgParams(kind="li_adaptive"), downscale=(2, 2)
)
c = dim3["C"]
g = dim3["G"]
r = dim3["R"]

In [ ]:
bc = bgv3["C"]
bg = bgv3["G"][3]
br = bgv3["R"][3]

In [ ]:
skimage.io.imshow(dim3["R"][1])

In [ ]:
plt.plot(dim3["R"][1][80, 30:])

In [ ]:
green = dim[0].compute()
bgg = segmentation.calculate_bg_iteratively(green)
bgg

In [ ]:
mask = segmentation.prob(green, bgg.bg, bgg.sd) > 0.01
green[mask]

In [ ]:
cyan = dim[2].compute()
bgc = segmentation.calculate_bg_iteratively(cyan)
mask = segmentation.prob(cyan, bgc.bg, bgc.sd) < 0.00000000000000000000001

In [ ]:
red = dim[3].compute()
red2 = dim[1].compute()

In [ ]:
plt.figure(figsize=(7.5, 7.5))

plt.subplot(3, 2, 1)
plt.hexbin(green[mask], cyan[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("green")
plt.ylabel("cyan")

plt.subplot(3, 2, 2)
plt.hexbin(red[mask], cyan[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("cyan")

ax = plt.subplot(3, 2, 4)
plt.hexbin(red[mask], green[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("green")

ax = plt.subplot(3, 2, 5)
plt.hexbin(red2[mask], green[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red2")
plt.ylabel("green")

ax = plt.subplot(3, 2, 6)
plt.hexbin(red2[mask], cyan[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red2")
plt.ylabel("cyan")

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 1)
plt.hexbin(g.ravel(), c.ravel(), bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("green")
plt.ylabel("cyan")

plt.subplot(2, 2, 2)
plt.hexbin(r.ravel(), c.ravel(), bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("cyan")

ax = plt.subplot(2, 2, 4)
plt.hexbin(r.ravel(), g.ravel(), bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("green")

axins = plt.axes([0.2, 0.12, 0.28, 0.28])
axins.hexbin(
    r.ravel(), g.ravel(), extent=(0, 80, 0, 150), bins="log", cmap=plt.cm.viridis_r
)

mark_inset(ax, axins, loc1=1, loc2=4, fc="none", ec="0.5")

In [ ]:
import statsmodels.api as sm

f = sm.qqplot_2samples(red[~mask], green[~mask])
f = sm.qqplot_2samples(red[~mask], cyan[~mask])

## flat image correction

In [ ]:
g1a = nima.read_tiff("/home/dati/dt-clop3/data/210917/3alpha/c1_2.tf8", channels)[0][
    "G"
][2]
g2a = nima.read_tiff("/home/dati/dt-clop3/data/210917/3alpha/c2_2.tf8", channels)[0][
    "G"
][2]
g3a = nima.read_tiff("/home/dati/dt-clop3/data/210917/3alpha/c3_2.tf8", channels)[0][
    "G"
][2]
g1 = nima.read_tiff("/home/dati/dt-clop3/data/210917/3/c1_2.tf8", channels)[0]["G"][2]
g2 = nima.read_tiff("/home/dati/dt-clop3/data/210917/3/c2_2.tf8", channels)[0]["G"][2]
g3 = nima.read_tiff("/home/dati/dt-clop3/data/210917/3/c3_2.tf8", channels)[0]["G"][2]

In [ ]:
plt.subplot(1, 2, 1)
skimage.io.imshow(
    skimage.filters.gaussian(np.max([g1, g2, g3, g1a, g2a, g3a], axis=0), sigma=500)
)
plt.subplot(1, 2, 2)
skimage.io.imshow(skimage.filters.gaussian(np.max([g1, g2, g3], axis=0), sigma=500))

## example w/out @Gain

In [ ]:
fp24 = "/home/dati/dt-clop3/data/210917/3/c1_2.tf8"

In [ ]:
dim, md = nima.read_tiffmd(fp24, channels=["G", "R", "C"])

In [ ]:
sns.histplot(dim[8, 0].ravel())
plt.yscale("log")

In [ ]:
utils.bg(dim[8, 0].compute())

In [ ]:
bgr = segmentation.calculate_bg_iteratively(dim[8, 1].compute())
bgr

In [ ]:
sns.histplot(
    dim[8, 1].compute()[
        (segmentation.prob(dim[8, 1].compute(), bgr.bg, bgr.sd) > 0.001)
    ]
)

In [ ]:
mask = (
    segmentation.prob(dim[8, 1].compute(), bgr.bg, bgr.sd)
    > 0.0000000000000000000000000000000000000000000000000000000000000000001
)
plt.imshow(~mask)

## BIAS and FLAT

TODO:
- pytest
- build a function callable from both the library and the CLI